In [1]:
import torch

In [2]:
model = torch.load('yolopv2.pt', weights_only=False)
model.eval()

C:\Users\Excalibur\anaconda3\envs\myenv\lib\site-packages\torch\serialization.py:799: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


RecursiveScriptModule(
  original_name=Model
  (model): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=Conv
      (conv): RecursiveScriptModule(original_name=Conv2d)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (1): RecursiveScriptModule(
      original_name=Conv
      (conv): RecursiveScriptModule(original_name=Conv2d)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (2): RecursiveScriptModule(
      original_name=Conv
      (conv): RecursiveScriptModule(original_name=Conv2d)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (3): RecursiveScriptModule(
      original_name=Conv
      (conv): RecursiveScriptModule(original_name=Conv2d)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (4): RecursiveScriptModule(
      original_name=Conv
      (conv): RecursiveScriptModule(original_name=Conv2d)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (5): Rec

In [2]:
import torch
import cv2
import numpy as np

# YOLOP modeli yükleme
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = torch.load('yolopv2.pt', weights_only=False)
model.eval()

# Giriş görüntüsü
image_path = "deneme.jpg"
img = cv2.imread(image_path)
org_img = img.copy()

# Görüntü ön işleme
def preprocess_image(frame):
    img = cv2.resize(frame, (640, 640))  # YOLOP için sabit boyut
    img = img[:, :, ::-1].transpose(2, 0, 1)  # HWC -> CHW
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).float()
    img /= 255.0  # Normalize
    img = img.unsqueeze(0)  # Batch dimension
    return img

# Görüntü işleme
input_img = preprocess_image(img).to(device)
img = cv2.resize(img, (640, 640))
# Model tahmini
with torch.no_grad():
    _, seg, ll = model(input_img)

ll = ll.squeeze().cpu().numpy()  # Tensor'dan NumPy'ye dönüşüm

# Segmentasyon çıktısını binary maskeye çevirme
binary_mask = (ll > 0.5).astype(np.uint8)
cv2.imshow('Binary Mask', binary_mask * 255)  # 255 ile maskeyi göstermek daha kolay olabilir.
cv2.waitKey(0)
cv2.destroyAllWindows()

# Şerit konturlarını bulma
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Görüntü boyutlarını alma
height, width, _ = img.shape
mask_height, mask_width = binary_mask.shape
mean = mask_width/2

# Orta noktaları saklamak için bir liste
lanes = [[0,0], [0,0]]
close_lanes = [999, 999]

# Orta noktaları hesaplama ve çizme
for contour in contours:
    # Küçük konturları filtreleme
    if cv2.contourArea(contour) < 50:  # Alan eşiğini ayarla (isteğe göre)
        continue

    # Konturun tüm x ve y koordinatlarını al
    points = contour[:, 0, :]  # (N, 2) şeklinde
    min_x = np.mean(points[:, 0])  # x koordinatlarının ortalaması
    min_y = np.mean(points[:, 1])  # y koordinatlarının ortalaması
    midpoint_x = int(min_x * (width / mask_width))
    midpoint_y = int(min_y * (height / mask_height))
    cv2.circle(img, (midpoint_x, midpoint_y), 7, (0, 255, 0), -1)
    # Maskedeki koordinatları orijinal boyuta ölçeklendir
    #cx = int(mean_x * (width / mask_width))
    #cy = int(mean_y * (height / mask_height))
    print(int(abs(min_x - mean) + abs(min_y - mask_height)))
    if close_lanes[0] > int(abs(min_x - mean) + abs(min_y - mask_height)):
        close_lanes[1] = close_lanes[0]
        lanes[1] = lanes[0]
        close_lanes[0] = int(abs(min_x - mean) + abs(min_y - mask_height))
        lanes[0] = [min_x, min_y]
    elif close_lanes[1] > int(abs(min_x - mean) + abs(min_y - mask_height)):
        close_lanes[1] = int(abs(min_x - mean) + abs(min_y - mask_height))
        lanes[1] = [min_x, min_y]
## Kırmızı bir nokta çiz
##cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)
#
# Eğer en az 2 şerit bulunmuşsa, iki şeridin ortasını hesapla
#    # İlk iki şerit alınarak ortalama merkezi hesapla
midpoint_x = int((lanes[0][0] + lanes[1][0]) / 2)
midpoint_y = int((lanes[0][1] + lanes[1][1]) / 2)
midpoint_x = int(midpoint_x * (width / mask_width))
midpoint_y = int(midpoint_y * (height / mask_height))
#
## Orta noktayı yeşil bir daire ile göster
cv2.circle(img, (midpoint_x, midpoint_y), 7, (5, 5, 5), -1)
#cv2.circle(img, (0, 320), 7, (0, 255, 0), -1)
#cv2.circle(img, (320, 0), 7, (0, 255, 255), -1)

# Sonucu gösterme
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cuda
528
491
737
585
661
